# 12b

This lab continues the geometry theme from **12a**. Recall the concepts from **12a**.

Given a collection of points $P = \{ p_i : i = 1,2,\cdots, k\} \subset \mathbb R^n$ their **convex hull** will be denoted

$$C(P) = \{ t_1p_1 + t_2p_2 + \cdots + t_kp_k : 0 \leq t_i \leq 1 \ \forall i, t_1+t_2+\cdots+t_k = 1 \}.$$

If one removes the constraints $0 \leq t_i \leq 1 \ \forall i$ one gets a more familiar notion from linear
algebra

$$Aff(P) = \{ t_1p_1 + t_2p_2 + \cdots + t_kp_k : t_i \in \mathbb R, t_1 + \cdots + t_k = 1 \}$$

is called the **affine subspace** spanned by the points $P$.  In general, it is the smallest-dimensional affine-linear subspace of $\mathbb R^n$ that contains the points **P**. 

Recall how in **12a** you rewrote the equations 

$$ v = t_1p_1 + t_2p_2 + \cdots + t_kp_k, \hskip 1cm 1 = t_1 + t_2 + \cdots + t_k$$

as the system

$$A\cdot T = \vec v$$

where $A$ was $P$ augmented with a row of $1$'s along the bottom, and $\vec v$ is $v$ with a single $1$ augmented along the bottom.

----

## Ex.1

Write an algorithm **on_aff(P, v)** that takes as input a numpy $n \times k$ matrix **P** and a numpy $n \times 1$ vector $v$. The algorithm will determine whether or not $v$ is on the affine subspace of $\mathbb R^n$ spanned by **P**.  

If $v$ is on the affine subspace, have  **on_aff** return the coefficients **T**.  If not, return **None**.

I recommend using [**least squares**](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html).  i.e. solve for $\tilde T$ minimizing $||A T - \vec v||^2$. Once you have computed $\tilde T$, check if $||A\tilde T-\vec v||=0$.  This notation is from Lab 12a. 

----

**Comments:**

You are free to use the numpy library linked above, or solve the problem directly, much like our previous least-squares problems.

We give a sketch how one would apply least squares in this situation. The least squares technique tells us the minima $\tilde T$ satisfies

$$ A^T(A \tilde T - \vec v) = 0$$
(this requires a little calculus argument)

Unfortunately, if the point collection $P$ is large relative to the ambient dimension $n$, the matrix $A^TA$ is not invertible -- this is because there are many solutions to the above equation.  One way to bypass this difficulty is to only use variables $t_i$ corresponding to the columns of $A$ that form a basis for the column space, i.e. this amounts to setting redundant variables $t_i=0$. 

In [1]:
import numpy as np


In [ ]:
def on_aff(P, v):
    '''
    Input: 
        P: (n x k) numpy matrix
        v: (n x 1) numpy vector
    Output:
        Determine whether or not v is on the affine subspace of R^n spanned by P
        Return:
            T: Coefficient matrix, if v is on the affine subspace
            None: Otherwise
    '''
    

----

## Ex. 2

For this problem, we ask you to extend your solution to **Ex.1** to create a function **on_hull(P, v)** which takes as input:

1. **P** an arbitrary $n \times k$ numpy matrix, thought of as $k$ column vectors in $\mathbb R^n$. 

2. **v** an $n \times 1$ numpy matrix.  

Have **on_hull** return the coefficient matrix **T** if it exists, and **None** otherwise. The coefficient matrix would be the coefficients describing **v** as an affine linear combination of the column vectors of **P**.

I suggest first solving for $\tilde T$ using your solution to **Ex.1**. 

----

All solutions to the equations

$$t_1 p_1 + \cdots + t_k p_k = v, \ \ t_1+ \cdots + t_k=1$$

are of the form $T = \tilde T + w$ where $w \in NullSpace(A)$.  

If **v** is not on the affine subspace spanned by **P**, your algorithm would terminate with **None**. 

But if **v** is on the affine subspace spanned by **P**, you would then look for a solution $T$ where $0 \leq t_i \leq 1 \ \forall i$.  

In more geometric language, we are asking if

$$(\tilde T + NullSpace(A)) \cap [0,1]^k \neq \emptyset.$$

This is a standard problem in a subject called **linear programming**.  The basic idea is that since $NullSpace(A)$ is connected and unbounded (provided $Nullity(A)>0$), then **if** there is such an intersection, there must be an intersection on the boundary of $[0,1]^k$.  

At this point you are free to solve the problem using the suggestion below, or with the [**scipy.optimize.linprog**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html) routine.  The challenge in this case would be, how do we rephrase this problem to one the library can handle? 

Repeating the argument two paragraphs up inductively, if $Nullity(A)=j$ then $\tilde A + NullSpace(A))$ must intersect $[0,1]^k$ on a boundary facet of dimension $k-j$. 

The boundary facets of dimension $k-1$ are precisely $$\{0,1\} \times [0,1]^{k-1}, \ \ [0,1] \times \{0,1\} \times [0,1]^{k-2}, \ \ [0,1]^2 \times \{0,1\} \times [0,1]^{k-3} \ \ \cdots \ \ [0,1]^{k-1} \times \{0,1\}$$
More generally, there are $2^j {k \choose j}$ boundary facets of dimension $k-j$.  One obtains them by taking $j$ of the $[0,1]$ factors of $[0,1]^k$ and replacing $[0,1]$ by $\{0,1\}$.  

You can iterate through distinct $j$-tuples of numbers in **range(n)** using the **itertools** library, like so: **for I in itertools.combinations(range(n), r=j):**.

----

**In summary, here is our algorithm:**

(1) Compute a basis for the nullspace of **A**. Use numpy's **np.linalg.null_space**. Let $j$ be the dimension of the nullspace.

(2) Iterate over all the $2^j {k \choose j}$ facets of $[0,1]^k$ of dimension $(k-j)$. Call such a facet $F$, and look for  solutions to the equation:
$$(\tilde T + NullSpace(A)) \cap F \neq \emptyset$$

To turn this into a proper set of equations, recall that the facet $F$ of $[0,1]^k$ is defined by demanding that $j$ of the coordinates either equal $0$ or $1$.  Given that $\tilde T + NullSpace(A)$ is $j$-dimensional, this is $j$ linear equations in $j$ variables, i.e. we should typically expect a unique solution.  Provided there is such a solution, we need to check the remaining coordinates are in the interval $[0,1]$. 

Note that if **any** of these equations has a solution, then that is a valid $T$ to return to the user. If **none** of these equation have solutions, then you can return **None**.

----

To test your algorithm create a routine **plot_conv(n,j)** that computes **n** random points in $\mathbb R^2$, and another **j** random points in $\mathbb R^2$.  Plot the **n** random points. Use **red** for the points in the convex hull of the **j** points, and **yellow** for points not in the convex hull. 

If you have time (and the inclination) repeat in $\mathbb R^3$ using **PlotLy**'s **Scatter3d** feature.  Sample plots are included below for $n=1000$, $j=6$. 

![alt text](12b.demo.png)
![alt text](12b.demo2.png)